Please modify only the cells with

        ##################
        # IMPLEMENT HERE #
        ##################

# Setup

### Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#### Change directory to where this file is located ####
%cd 'COPY&PASTE FILE DIRECTORY HERE'

### Import Modules

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Multinomial Naive Bayes Classifier

### Model

In [184]:
class MultinomialNaiveBayes:
    def __init__(self, alpha=1.0):
        """Initializes the model hyperparameter alpha(smoothing factor).
        """
        self.classes = None
        self.priors = None
        self.probs = None
        self.alpha = alpha

    def fit(self, X, y):
        """Estimates prior of classes and conditional probabilities given each class.

        Args:
            X: a numpy array of training data, shape (N, feature_dim)
            y: a numpy array of training label classes, shape (N,)
        """
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        self.priors = np.zeros(n_classes, dtype=np.float64)
        self.probs = np.zeros((n_classes, n_features), dtype=np.float64)

        #### Question (a) #####
        # Estimate self.priors and self.probs.
        # You must utilize self.alpha (smoothing factor)

        ##################
        # IMPLEMENT HERE #
        ##################
        for k in range(n_classes):
            self.priors[k] = self.alpha + np.sum(y == k)
        self.priors /= (n_classes * self.alpha + n_samples)

        X_sum = np.sum(X, axis=1)
        for k in range(n_classes):
            for j in range(n_features):
                self.probs[k,j] = (self.alpha + np.sum(X[y==k, j]))/ (n_features * self.alpha + np.sum(np.int64(y == k) * X_sum))

    def predict(self, X):
        """Predicts the class of input X.

        Args:
            X: a numpy array of test data, shape (N, feature_dim)

        Returns:
            y_hat: a numpy array of predicted classes, shape (N,)
        """

        #### Question (b) #####
        # Predict the class of input X using estimated self.priors and self.probs.
        # As we learned in class, it is numerically stable to use log probability instead of raw probability.

        ##################
        # IMPLEMENT HERE #
        ##################
        N,_ = X.shape
        y_hat = np.zeros(N,dtype = np.float64)
        for n in range(N):
            y_hat[n] = np.argmax(np.log(self.priors) + np.sum(np.log(self.probs)*X[n,:], axis=1))
        return y_hat
       

# Log Classification with Multinomial NB Classifier

### Load Dataset

In [161]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [162]:
pd.set_option('display.max_colwidth', None)
train.head()

,full_log,level
0,"Jan 13 07:53:27 localhost kibana: {""type"":""log"",""@timestamp"":""2021-01-12T22:53:27Z"",""tags"":[""error"",""task_manager""],""pid"":4332,""message"":""Failed to poll for work: Error: No Living connections""}",0
1,"type=SYSCALL msg=audit(1611884598.851:46637): arch=c000003e syscall=2 success=yes exit=10 a0=7fd5641bd552 a1=80000 a2=1b6 a3=24 items=1 ppid=1 pid=16168 auid=1000 uid=0 gid=980 euid=0 suid=0 fsuid=0 egid=980 sgid=980 fsgid=980 tty=(none) ses=1 comm=""ossec-syscheckd"" exe=""/var/esild/bin/ossec-syscheckd"" subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1611884598.851:46637): cwd=""/"" type=PATH msg=audit(1611884598.851:46637): item=0 name=""/etc/passwd"" inode=35395316 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:passwd_file_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PROCTITLE msg=audit(1611884598.851:46637): proctitle=""/var/esild/bin/ossec-syscheckd""",1
2,"Jan 5 07:23:42 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-04T22:23:42Z"",""tags"":[""warning"",""stats-collection""],""pid"":4820,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}",0
3,System Audit: SSH Hardening - 3: Root can log in. File: /etc/ssh/sshd_config. Reference: 3 .,1
4,Oct 30 18:35:00 localhost suricata[1444]: [1:2010937:3] ET SCAN Suspicious inbound to mySQL port 3306 [Classification: Potentially Bad Traffic] [Priority: 2] {TCP} 192.168.0.160:44768 -> 192.168.0.42:3306,0


### Preprocess Dataset

In [163]:
vectorizer = CountVectorizer(analyzer="word", max_features=1000)
vectorizer.fit(train['full_log'])

CountVectorizer(max_features=1000)

In [164]:
print('Index for Word "kibana":', vectorizer.vocabulary_["kibana"])
print('Index for Word "error":', vectorizer.vocabulary_["error"])
print('Index for Word "localhost":', vectorizer.vocabulary_["localhost"])

Index for Word "kibana": 717
Index for Word "error": 633
Index for Word "localhost": 736


### Train and Test Log Classifier

In [165]:
train_X = vectorizer.transform(train['full_log'])
train_y = train['level'].values
test_X = vectorizer.transform(test['full_log'])
test_y = test['level'].values

In [206]:
#### Question (c) #####
# Create an instance using class MultinomialNaiveBayes with alpha = 1.
# Then fit the model with training data(train_X, train_y), and calcuate accuracy using test data(test_X, test_y).

##################
# IMPLEMENT HERE #
##################
model = MultinomialNaiveBayes(1)
model.fit(train_X.toarray(),train_y)
y_hat = model.predict(test_X.toarray())
acc = np.sum(test_y == y_hat)/len(test_y)
print("Accuracy:", acc)

Accuracy: 0.9789827569801985


In [197]:
#### Question (d) ####
# (HINT) You can exploit these variables.
# No implementation needed.

word_idx = vectorizer.vocabulary_["ossec"]
security_level = 2

In [207]:
prob_ossec_given_2 = model.probs[security_level, word_idx]
print("Probability of 'ossec' given level 2:", prob_ossec_given_2)

Probability of 'ossec' given level 2: 1.1480660826837192e-05


In [204]:
model = MultinomialNaiveBayes(0)
model.fit(train_X.toarray(),train_y)
prob_ossec_given_2 = model.probs[security_level, word_idx]
print("Probability of 'ossec' given level 2:", prob_ossec_given_2)

Probability of 'ossec' given level 2: 0.0
